In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

%matplotlib notebook  

In [1]:



import os
import sys
import numpy 

import torch
import torch.nn as nn

import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.spatial import Delaunay
from collections import OrderedDict

import pymeshlab
from pytorch3d.ops import knn_points
from preprocess import find_neighbor
from layers import SpatialDescriptor, StructuralDescriptor, MeshConvolution

import open3d as o3d
import trimesh
from scipy.spatial import KDTree

from pytorch3d.io import save_obj
from pytorch3d.loss import chamfer_distance
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.structures import Meshes
from pytorch3d.io import load_obj, save_obj
from pytorch3d.datasets import collate_batched_meshes,

from dgcnn_model import DGCNN


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as Fu
from torch.utils.data import Dataset, DataLoader

# PyTorch3D imports for mesh handling and I/O.
from pytorch3d.structures import Meshes
from pytorch3d.io import load_objs_as_meshes

# =============================================================================
# 1. Dataset and DataLoader
# =============================================================================
import os
import torch
from torch.utils.data import Dataset
from pytorch3d.io import load_objs_as_meshes

class MeshDataset(Dataset):
    """
    A dataset that loads matching pairs of meshes from two folders.
    Assumes that both folders contain OBJ files with the same names (or sorted order).
    For example, "mesh_000.obj" in the bad folder corresponds to "mesh_000.obj" in the good folder.
    """
    def __init__(self, bad_folder, good_folder, device=None):
        """
        Args:
            bad_folder (str): Path to the folder containing meshes with bad topology.
            good_folder (str): Path to the folder containing meshes with good topology.
            device (torch.device, optional): Device on which to load the meshes.
        """
        self.bad_folder = bad_folder
        self.good_folder = good_folder

        # List and sort the files (assuming the naming convention makes them correspond)
        self.bad_files = sorted([os.path.join(bad_folder, f)
                                 for f in os.listdir(bad_folder) if f.endswith('.obj')])
        self.good_files = sorted([os.path.join(good_folder, f)
                                  for f in os.listdir(good_folder) if f.endswith('.obj')])
        
        # Make sure the dataset sizes match.
        assert len(self.bad_files) == len(self.good_files), "Mismatched number of files in folders"
        
        self.device = device if device is not None else \
                      torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def __len__(self):
        return len(self.bad_files)

    def __getitem__(self, idx):
        # Load the "bad" mesh and the corresponding "good" mesh.
        # load_objs_as_meshes returns a Meshes object with a batch size of 1.

        bad_mesh = load_objs_as_meshes([self.bad_files[idx]], device=self.device, load_textures=False)
        good_mesh = load_objs_as_meshes([self.good_files[idx]], device=self.device, load_textures=False)

 


        
        bad_faces, bad_verts = convert_tris_to_quads(bad_mesh)
        good_faces, good_verts = convert_tris_to_quads(good_mesh)
        # Normalize the vertices.
        


        bad_verts_norm = normalize_vertices(bad_verts)
        good_verts_norm = normalize_vertices(good_verts)
        # Create and return a new Meshes object with normalized vertices.
        normalized_bad_mesh = Meshes(verts=[bad_verts_norm], faces=[bad_faces])
        normalized_good_mesh = Meshes(verts=[good_verts_norm], faces=[good_faces])
        
        return normalized_bad_mesh, normalized_good_mesh

def convert_tris_to_quads(mesh):
    """
    Convert a PyTorch3D Mesh (with triangulated faces) back to a single face tensor
    that includes both quad and triangle faces. Triangle faces are padded by repeating
    the last vertex so that all faces have four indices.

    Args:
        mesh (pytorch3d.structures.Meshes): The PyTorch3D mesh object.
        device (str): The device to which the output tensors will be moved.

    Returns:
        faces (torch.Tensor): Tensor of face indices with shape (F, 4). For triangle faces,
                              the last index is repeated.
        verts (torch.Tensor): Tensor of vertex positions of shape (V, 3).
    """
    # Extract vertices and triangle faces
    verts = mesh.verts_list()[0]      # (V, 3)
    tri_faces = mesh.faces_list()[0]    # (F, 3)

    # Build a mapping from each sorted edge to the triangle face indices that contain it.
    edge_to_faces = {}
    for i, face in enumerate(tri_faces):
        edges = [
            (face[0].item(), face[1].item()),
            (face[1].item(), face[2].item()),
            (face[2].item(), face[0].item()),
        ]
        for edge in edges:
            edge = tuple(sorted(edge))
            edge_to_faces.setdefault(edge, []).append(i)

    # Merge adjacent triangles into quads if possible.
    used_tris = set()
    merged_quads = []
    for edge, faces in edge_to_faces.items():
        if len(faces) == 2:
            f1, f2 = faces
            if f1 in used_tris or f2 in used_tris:
                continue  # already merged into a quad

            # Combine vertices from both triangles.
            verts_f1 = tri_faces[f1].tolist()
            verts_f2 = tri_faces[f2].tolist()
            unique_vertices = list(set(verts_f1 + verts_f2))
            if len(unique_vertices) == 4:
                merged_quads.append(unique_vertices)
                used_tris.update([f1, f2])

    # For triangles that were not merged, pad them to have 4 vertices.
    remaining_tris = []
    for i in range(len(tri_faces)):
        if i not in used_tris:
            tri = tri_faces[i].tolist()
            padded_tri = tri + [tri[-1]]  # pad by repeating the last vertex
            remaining_tris.append(padded_tri)

    # Combine merged quads and padded triangles.
    all_faces = merged_quads + remaining_tris
    faces_tensor = torch.tensor(all_faces, dtype=torch.long, device=device)
    verts = verts.to(device)

    return faces_tensor, verts



def export_mesh_to_obj(verts, faces, filename):
    """
    Exports a mesh to an OBJ file.
    
    Args:
        verts (Tensor): A (V, 3) tensor of vertex positions.
        faces (Tensor): A (F, N) tensor of face indices (N = 3 for triangles, 4 for quads, etc.).
        filename (str): Output file path.
    """
    with open(filename, "w") as f:
        # Write vertices.
        for v in verts:
            # v[0], v[1], v[2] are the x, y, z coordinates.
            f.write(f"v {v[0].item()} {v[1].item()} {v[2].item()}\n")
        
        # Write faces.
        for face in faces:
            # OBJ format is 1-indexed.
            face_idx = [str(int(idx.item()) + 1) for idx in face]
            f.write("f " + " ".join(face_idx) + "\n")

    print(f"Mesh exported to {filename}")

def normalize_vertices(verts):
    """
    Normalize vertices uniformly so that the mesh fits within [-1, 1] in all directions
    while preserving proportions.
    """
    # Compute the center of the vertices
    center = verts.mean(dim=0, keepdim=True)
    verts_centered = verts - center

    # Find the maximum absolute distance along any axis (or overall Euclidean norm)
    # Option A: Use maximum absolute value across all axes.
    max_val = verts_centered.abs().max()
    
    # Option B: Use the maximum Euclidean distance from the center.
    # max_val = torch.norm(verts_centered, dim=1).max()

    # Avoid division by zero
    if max_val == 0:
        max_val = 1.0

    # Uniformly scale the vertices so that they lie in [-1, 1]
    verts_norm = verts_centered / max_val
    return verts_norm
    
def collate_meshes(batch):
    """
    Collates a list of (bad_mesh, good_mesh) tuples into a single tuple of batched Meshes:
      (batched_bad_meshes, batched_good_meshes).
      
    Args:
        batch (List[Tuple[Meshes, Meshes]]): Each element is a tuple containing two Meshes objects.
        
    Returns:
        Tuple[Meshes, Meshes]: Batched Meshes for bad and good meshes, respectively.
    """
    # Each sample in batch is a tuple (bad_mesh, good_mesh)
    # Extract vertex and face lists from each Meshes object.
    bad_verts = [sample[0].verts_list()[0] for sample in batch]
    bad_faces = [sample[0].faces_list()[0] for sample in batch]
    good_verts = [sample[1].verts_list()[0] for sample in batch]
    good_faces = [sample[1].faces_list()[0] for sample in batch]
    
    batched_bad_meshes = Meshes(verts=bad_verts, faces=bad_faces)
    batched_good_meshes = Meshes(verts=good_verts, faces=good_faces)
    return batched_bad_meshes, batched_good_meshes





New Generator
==============





In [3]:
class GraphConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphConv, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)
        
    def forward(self, x, edge_index):
        """
        Args:
            x: Tensor of shape (V, in_channels), vertex features.
            edge_index: Tensor of shape (2, E), where each column is (source, target).
        Returns:
            Tensor of shape (V, out_channels).
        """
        V = x.size(0)
        # Using torch_scatter to aggregate messages from neighbors.
        import torch_scatter
        src = edge_index[0]  # source vertices (E,)
        target = edge_index[1]  # target vertices (E,)
        messages = x[src]      # (E, in_channels)
        # Aggregate (mean) messages for each target vertex.
        agg = torch_scatter.scatter_mean(messages, target, dim=0, dim_size=V)
        return self.linear(agg)

# -------------------------------
# Helper: Compute Edge Index from Faces
# -------------------------------
def compute_edge_index(face_list):
    """
    Given a list of faces (each face is a tensor or list of vertex indices),
    computes an edge index tensor (2, E). For each face, consecutive vertices
    (including the last connecting to the first) produce undirected edges.
    """
    edges = []
    for face in face_list:
        # Convert to list if necessary.
        face_list_val = face.tolist() if isinstance(face, torch.Tensor) else face
        num_slots = len(face_list_val)
        for j in range(num_slots):
            src = face_list_val[j]
            tgt = face_list_val[(j + 1) % num_slots]
            edges.append((src, tgt))
            edges.append((tgt, src))  # undirected edge
    if len(edges) > 0:
        edge_index = torch.tensor(edges, dtype=torch.long)
        edge_index = edge_index.t().contiguous()  # shape: (2, E)
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    return edge_index

# -------------------------------
# Adapted MeshGenerator Module
# -------------------------------
class MeshGenerator(nn.Module):
    def __init__(self, hidden_dim=64, tau=1.0):
        super(MeshGenerator, self).__init__()
        self.hidden_dim = hidden_dim
        self.tau = tau

        # Initial per-vertex feature extractor.
        self.vertex_mlp = nn.Sequential(
            nn.Linear(3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Graph convolution layer to incorporate local neighborhood context.
        self.graph_conv = GraphConv(hidden_dim, hidden_dim)
        
        # Predict vertex offsets (initialized to near zero).
        self.offset_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)
        )
        nn.init.constant_(self.offset_mlp[-1].weight, 0.0)
        nn.init.constant_(self.offset_mlp[-1].bias, 0.0)
        
        # Shared predictor for face connectivity. It takes a concatenated feature
        # of the face-level feature and the original vertex feature (size: 2*hidden_dim)
        # and produces a query vector (size: hidden_dim).
        self.shared_face_predictor = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
    def forward(self, meshes: Meshes):
        
        new_verts_list = []
        new_faces_list = []
        
        # Process each mesh in the batch.
        for verts, faces in zip(meshes.verts_list(), meshes.faces_list()):
            # verts: (V, 3)
            V = verts.shape[0]
            
            # Compute per-vertex features.
            v_feats = self.vertex_mlp(verts)  # (V, hidden_dim)           
            # Compute graph-based aggregation.
            # Compute an edge_index from the face connectivity (faces is a list of tensors).
            edge_index = compute_edge_index(faces)
            edge_index = edge_index.to(device)
            # Refine vertex features by combining original features with aggregated info.
            refined_feats = v_feats + self.graph_conv(v_feats, edge_index)  # (V, hidden_dim)
            
            # Predict vertex offsets using the refined features.
            offsets = self.offset_mlp(refined_feats)  # (V, 3)
            new_verts = verts + offsets               # (V, 3)
            
            # Process connectivity: new_face_indices will hold new connectivity for each face.
            new_face_indices = []
            for idx, face in enumerate(faces):
                # Each face is a tensor (or list) of vertex indices; num_slots can vary.
                num_slots = face.shape[0]
                # Compute face-level feature as mean over the refined vertex features.
                face_v_feats = refined_feats[face]  # (num_slots, hidden_dim)
                face_feat = face_v_feats.mean(dim=0, keepdim=True)  # (1, hidden_dim)
                new_face = []
                for i in range(num_slots):
                    # Get the original vertex feature for slot i (from refined features).
                    orig_v_feat = refined_feats[face[i]].unsqueeze(0)  # (1, hidden_dim)
                    # Concatenate face-level feature with the original vertex feature.
                    combined_feat = torch.cat([face_feat, orig_v_feat], dim=1)  # (1, 2*hidden_dim)
                    # Compute query vector using the shared face predictor.
                    query = self.shared_face_predictor(combined_feat)  # (1, hidden_dim)
                    
                    # --- k-NN Restriction to Reduce Candidate Set ---
                    # Use the original vertex position (from verts) as the reference.
                    ref_point = verts[face[i]].unsqueeze(0)  # (1, 3)
                    query_points = ref_point.unsqueeze(1)     # (1, 1, 3)
                    verts_expanded = verts.unsqueeze(0)         # (1, V, 3)
                    # Use K=10; adjust as needed.
                    knn_out = knn_points(query_points, verts_expanded, K=2)
                    # knn_out.idx: (1, 1, 10) → squeeze to (1, 10)
                    knn_indices = knn_out.idx.squeeze(1)  # (1, 10)
                    
                    # Gather candidate features from the refined features.
                    candidate_features = refined_feats[knn_indices]  # (1, 10, hidden_dim)
                    
                    # --- Compute Candidate Scores ---
                    # Unsqueeze query to (1, 1, hidden_dim) and perform batch matrix multiplication.
                    query_unsq = query.unsqueeze(1)  # (1, 1, hidden_dim)
                    candidate_scores = torch.bmm(query_unsq, candidate_features.transpose(1, 2)).squeeze(1)  # (1, 10)
                    
                    # --- Biasing ---
                    # Create a mask: check if candidate equals the original vertex index.
                    match = (knn_indices == face[i].unsqueeze(0))  # (1, 10), bool
                    bias = torch.where(match, torch.tensor(3.0, device=verts.device),
                                         torch.tensor(-1.0, device=verts.device))
                    candidate_scores = candidate_scores + bias
                    
                    # --- Differentiable Selection (Gumbel-softmax) ---
                    face_slot_onehot = F.gumbel_softmax(candidate_scores, tau=self.tau, hard=True, dim=1)  # (1, 10)
                    chosen_idx_in_candidates = face_slot_onehot.argmax(dim=1)  # (1,)
                    # Map back to actual vertex index.
                    chosen_idx = knn_indices[0, chosen_idx_in_candidates.item()]
                    new_face.append(chosen_idx)
                # Convert new_face (list of ints) to tensor.
                new_face_tensor = torch.tensor(new_face, device=verts.device)
                new_face_indices.append(new_face_tensor)
                #print(f"Processed face {idx+1}/{len(faces)}")
            new_verts_list.append(new_verts)
            new_faces_list.append(new_face_indices)
        if len(new_faces_list) == 1 and isinstance(new_faces_list[0], list):
            faces_inner = new_faces_list[0]
        else:
            faces_inner = new_faces_list

        # Convert each face from a list of tensors to a list of integers.
        faces_list = []
        # Check if we have a list of lists (i.e., batch size > 1)
        if isinstance(faces_inner[0], list):
            # faces_inner is a list where each element is a list of face tensors from a mesh.
            for mesh_faces in faces_inner:
                for face in mesh_faces:
                    new_face = [int(v.item()) for v in face]
                    faces_list.append(new_face)
        else:
            # Single mesh case
            for face in faces_inner:
                new_face = [int(v.item()) for v in face]
                faces_list.append(new_face)

        new_faces_list_out = []
        for mesh_faces in new_faces_list:  # new_faces_list is a list where each element is a list of face tensors for one mesh.
            # Convert each face tensor to a long tensor (if not already) and stack them.
            # Each mesh_faces is a list of tensors with face indices.
            faces_tensor = torch.stack([face.long() for face in mesh_faces], dim=0)  # shape: (#faces, face_length)
            new_faces_list_out.append(faces_tensor)

        # Return the Meshes object with vertices and connectivity preserved per mesh.
        return Meshes(verts=new_verts_list, faces=new_faces_list_out)

New Discriminator
==============





In [4]:
class GraphConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphConv, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)
        
    def forward(self, x, edge_index):
        """
        Args:
            x: Tensor of shape (V, in_channels) -- per-vertex features.
            edge_index: Tensor of shape (2, E) where each column is (source, target).
        Returns:
            Tensor of shape (V, out_channels) with aggregated neighbor features.
        """
        V = x.size(0)
        # We'll do a simple mean aggregation over neighbors.
        # Initialize an accumulator and a count.
        agg = torch.zeros_like(x)
        count = torch.zeros(V, device=x.device)
        # edge_index[0] are source vertices, edge_index[1] are target vertices.
        for src, tgt in edge_index.t():
            agg[tgt] += x[src]
            count[tgt] += 1
        # Avoid division by zero
        count = count.unsqueeze(1).clamp(min=1)
        agg = agg / count
        return self.linear(agg)

# -------------------------------
# Helper: Compute Edge Index from Faces
# -------------------------------
def compute_edge_index(faces):
    """
    Given a tensor of faces of shape (F, num_slots) (e.g., (F,3) for triangles or (F,4) for quads),
    computes an edge index tensor (2, E). For each face, consecutive vertices (including last to first)
    form undirected edges.
    """
    F, num_slots = faces.shape
    edges = []
    for f in range(F):
        face = faces[f]
        # For each face, add edges (bidirectional)
        for i in range(num_slots):
            src = face[i].item()
            tgt = face[(i+1) % num_slots].item()
            edges.append((src, tgt))
            edges.append((tgt, src))
    if len(edges) > 0:
        edge_index = torch.tensor(edges, dtype=torch.long, device=faces.device)
        edge_index = edge_index.t().contiguous()  # shape: (2, E)
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long, device=faces.device)
    return edge_index

# -------------------------------
# Adapted MeshDiscriminator Module
# -------------------------------
class MeshDiscriminator(nn.Module):
    """
    A conditional discriminator that takes a pair of meshes:
      - candidate: the mesh to judge (either generated or a real mesh with good topology)
      - reference: the good topology mesh as a reference.
      
    This version focuses on topology by refining vertex features using a graph convolution
    based on the connectivity of the mesh.
    """
    def __init__(self, hidden_dim=64):
        super(MeshDiscriminator, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Shared per-vertex feature extractor.
        self.vertex_mlp = nn.Sequential(
            nn.Linear(3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Graph convolution layer to incorporate neighborhood connectivity.
        self.graph_conv = GraphConv(hidden_dim, hidden_dim)
        
        # Process face features.
        self.face_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Global projection.
        self.global_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Final classifier: combine candidate and reference global topology features.
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
    def extract_global_feature(self, mesh: Meshes):
        """
        Extracts a global feature vector from a mesh based primarily on its connectivity.
        Assumes mesh is a batch of one.
        """
        # Get vertices and faces. Faces is assumed to be a tensor of shape (F, num_slots).
        verts = mesh.verts_list()[0]  # (V, 3)
        faces = mesh.faces_list()[0]  # (F, num_slots) -- works for triangles, quads, etc.
        
        # Compute initial per-vertex features.
        v_feats = self.vertex_mlp(verts)  # (V, hidden_dim)
        # Refine features with graph convolution based on connectivity.
        edge_index = compute_edge_index(faces)  # (2, E)
        refined_feats = v_feats + self.graph_conv(v_feats, edge_index)  # (V, hidden_dim)
        
        # For each face, gather the refined vertex features.
        # refined_face_feats: (F, num_slots, hidden_dim)
        refined_face_feats = refined_feats[faces]
        # Aggregate face features by averaging over the vertices in the face.
        face_feats = refined_face_feats.mean(dim=1)  # (F, hidden_dim)
        face_feats = self.face_mlp(face_feats)         # Further processing.
        # Global feature: average over faces.
        global_feat = face_feats.mean(dim=0)  # (hidden_dim,)
        global_feat = self.global_mlp(global_feat)  # (hidden_dim,)
        return global_feat
    
    def forward(self, candidate_meshes: Meshes, ref_meshes: Meshes):
        """
        For each mesh pair (candidate, reference), extracts global topology features and
        uses the classifier to predict whether the candidate mesh exhibits good topology.
        """
        
        batch_size = len(candidate_meshes.verts_list())
        cand_features = []
        ref_features = []
        # Process each pair of meshes using their own connectivity.
        for cand_mesh, ref_mesh, cand_faces, ref_faces in zip(
                candidate_meshes.verts_list(),
                ref_meshes.verts_list(),
                candidate_meshes.faces_list(),
                ref_meshes.faces_list()):
            cand = Meshes(verts=[cand_mesh], faces=[cand_faces])
            ref  = Meshes(verts=[ref_mesh],  faces=[ref_faces])
            cand_features.append(self.extract_global_feature(cand))
            ref_features.append(self.extract_global_feature(ref))
        cand_features = torch.stack(cand_features, dim=0)  # (B, hidden_dim)
        ref_features  = torch.stack(ref_features, dim=0)     # (B, hidden_dim)
        combined = torch.cat([cand_features, ref_features], dim=1)  # (B, 2*hidden_dim)
        out = self.classifier(combined)  # (B, 1)
        
        return out

New Loss Functions
==============


In [5]:
def convexity_loss(verts, faces, tol=0.0):
    """
    Computes a convexity loss for a single mesh.
    
    Args:
        verts (Tensor): (V, 3) tensor of vertex positions.
        faces (Tensor): (F, 3) tensor of face indices (for triangles).
        tol (float): Tolerance; values above this are considered valid.
        
    Returns:
        Tensor: A scalar loss. Zero if every vertex lies on or outside every face.
    """
    # Retrieve vertices for each face
    v0 = verts[faces[:, 0]]  # (F, 3)
    v1 = verts[faces[:, 1]]  # (F, 3)
    v2 = verts[faces[:, 2]]  # (F, 3)
    
    # Compute face centers and normals.
    face_center = (v0 + v1 + v2) / 3.0  # (F, 3)
    face_normal = torch.cross(v1 - v0, v2 - v0)  # (F, 3)
    face_normal = F.normalize(face_normal, dim=1)  # (F, 3)
    
    # For each face, compute signed distances for all vertices.
    # Expand dimensions so we can compute pairwise differences:
    # face_center: (F, 1, 3) and verts: (1, V, 3) → (F, V, 3)
    diff = verts.unsqueeze(0) - face_center.unsqueeze(1)  # (F, V, 3)
    
    # Compute dot product along the last dimension: (F, V)
    dot = torch.sum(diff * face_normal.unsqueeze(1), dim=2)
    
    # We want dot >= tol for a convex mesh. Any negative deviation is penalized.
    penalty = F.relu(tol - dot)  # (F, V)
    
    # Average penalty over all faces and vertices.
    loss = penalty.mean()
    return loss

New Training
==============





In [ ]:
num_epochs = 30
lambda_chamfer = 1
lambda_convexity = 1
batch_size = 4

bad_folder = "newModelData/badTopo"
good_folder = "newModelData/goodTopo"

# --- Create Dataset and DataLoader ---
dataset = MeshDataset(bad_folder, good_folder)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_meshes)

# Initialize models and move them to device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
#torch.set_printoptions(threshold=sys.maxsize)

generator = MeshGenerator(hidden_dim=64, tau=1).to(device)
discriminator = MeshDiscriminator(hidden_dim=64).to(device)

# Set up optimizers.
optimizer_g = optim.Adam(generator.parameters(), lr=1e-4)
optimizer_d = optim.Adam(discriminator.parameters(), lr=1e-4)

# Define adversarial loss.
adversarial_loss = nn.BCELoss()



for epoch in range(num_epochs):
    for i, (bad_meshes, good_meshes) in enumerate(dataloader):
        bad_meshes = bad_meshes.to(device)
        good_meshes = good_meshes.to(device)
        batch_size = len(bad_meshes.verts_list())

        valid = torch.ones((batch_size, 1), device=device)
        fake = torch.zeros((batch_size, 1), device=device)
        
        # -------------------------
        #  Train Discriminator
        # -------------------------
        optimizer_d.zero_grad()

        
        # Real pairs: discriminator sees (good, good)
        pred_real = discriminator(good_meshes, good_meshes)
        loss_real = adversarial_loss(pred_real, valid)

        # Generate fake meshes and detach to avoid propagating gradients to generator.
        fake_meshes = generator(bad_meshes)
        fake_meshes_detached = fake_meshes.detach()
        pred_fake = discriminator(fake_meshes_detached, good_meshes)
        loss_fake = adversarial_loss(pred_fake, fake)
        
        loss_d = (loss_real + loss_fake) / 2
        loss_d.backward()
        optimizer_d.step()
        
        # -------------------------
        #  Train Generator
        # -------------------------
        optimizer_g.zero_grad()
        
        # Recompute fake meshes so that the generator graph is intact.
        fake_meshes_for_gen = generator(bad_meshes)
        pred_fake_for_g = discriminator(fake_meshes_for_gen, good_meshes)
        loss_g_adv = adversarial_loss(pred_fake_for_g, valid)

        # Compute Chamfer loss to preserve shape with good_meshes instead of bad_meshes
        chamfer_loss_total = 0.0
        fake_verts_list = fake_meshes_for_gen.verts_list()
        good_verts_list = good_meshes.verts_list()
        for fake_verts, good_verts in zip(fake_verts_list, good_verts_list):
            loss, _ = chamfer_distance(fake_verts.unsqueeze(0), good_verts.unsqueeze(0))
            chamfer_loss_total += loss
        chamfer_loss = chamfer_loss_total / len(fake_verts_list)

        convexity_loss_total = 0.0
        for verts, faces in zip(fake_meshes_for_gen.verts_list(), fake_meshes_for_gen.faces_list()):
            convexity_loss_total += convexity_loss(verts, faces, tol=0.0)
        convexity_loss_avg = convexity_loss_total / len(fake_meshes_for_gen.verts_list())

        # Combine losses.
        loss_g = loss_g_adv + lambda_chamfer * chamfer_loss + lambda_convexity * convexity_loss_avg
        loss_g.backward()
        optimizer_g.step()
        
        # Optionally, print training progress.
        print(f"[Epoch {epoch+1}/{num_epochs}] [Batch {i+1}/{len(dataloader)}] "
              f"Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}")

cuda


C:\Users\Ben\AppData\Local\Temp\ipykernel_4252\1287052893.py:20: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Cross.cpp:66.)
  face_normal = torch.cross(v1 - v0, v2 - v0)  # (F, 3)


[Epoch 1/30] [Batch 1/6] Loss D: 0.6932, Loss G: 0.9343
[Epoch 1/30] [Batch 2/6] Loss D: 0.6932, Loss G: 0.9219
[Epoch 1/30] [Batch 3/6] Loss D: 0.6931, Loss G: 0.9250
[Epoch 1/30] [Batch 4/6] Loss D: 0.6931, Loss G: 1.0611
[Epoch 1/30] [Batch 5/6] Loss D: 0.6931, Loss G: 0.9773
[Epoch 1/30] [Batch 6/6] Loss D: 0.6931, Loss G: 1.0493
[Epoch 2/30] [Batch 1/6] Loss D: 0.6931, Loss G: 0.9402
[Epoch 2/30] [Batch 2/6] Loss D: 0.6931, Loss G: 1.0221
[Epoch 2/30] [Batch 3/6] Loss D: 0.6931, Loss G: 0.9119
[Epoch 2/30] [Batch 4/6] Loss D: 0.6931, Loss G: 1.0174
[Epoch 2/30] [Batch 5/6] Loss D: 0.6931, Loss G: 0.9571
[Epoch 2/30] [Batch 6/6] Loss D: 0.6931, Loss G: 0.9839
[Epoch 3/30] [Batch 1/6] Loss D: 0.6931, Loss G: 0.9898
[Epoch 3/30] [Batch 2/6] Loss D: 0.6931, Loss G: 0.9941
[Epoch 3/30] [Batch 3/6] Loss D: 0.6930, Loss G: 0.9490
[Epoch 3/30] [Batch 4/6] Loss D: 0.6930, Loss G: 0.9801
[Epoch 3/30] [Batch 5/6] Loss D: 0.6931, Loss G: 0.9197
[Epoch 3/30] [Batch 6/6] Loss D: 0.6931, Loss G:

Output
==============





In [7]:
def triangulate_faces(face_list):
    """
    Converts a list of faces (which may have variable lengths) to a tensor of triangular faces.
    Each face is represented as a list or 1D tensor of vertex indices.
    
    Args:
        face_list (List[Union[List[int], Tensor]]): List of faces.
        
    Returns:
        Tensor: Triangular faces of shape (num_triangles, 3).
    """
    tri_faces = []
    for face in face_list:
        # Convert tensor to list if necessary.
        face_indices = face.tolist() if isinstance(face, torch.Tensor) else face
        if len(face_indices) == 3:
            tri_faces.append(face_indices)
        elif len(face_indices) >= 3:
            # Use fan triangulation: for a face [a, b, c, d, ...],
            # create triangles [a, b, c], [a, c, d], etc.
            for i in range(1, len(face_indices)-1):
                tri_faces.append([face_indices[0], face_indices[i], face_indices[i+1]])
    return torch.tensor(tri_faces, dtype=torch.long)

# Example usage:
# Suppose new_meshes is your Meshes object output by the generator.
# Each mesh has new_verts_list and new_faces_list, where new_faces_list is a list of face lists (variable-length).
for i, (verts, faces) in enumerate(zip(fake_meshes_for_gen.verts_list(),fake_meshes_for_gen.faces_list())):
    # Triangulate faces if they are not already triangles.
    # Here, faces is assumed to be a list of faces (each face a list or 1D tensor).
    tri_faces = triangulate_faces(faces)
    
    filename = f"output_mesh_{i:03d}.obj"
    save_obj(filename, verts, tri_faces)
    print(f"Saved mesh {i} to {filename}")

Saved mesh 0 to output_mesh_000.obj
